#  Модель Transformer
Задание
Реализовать суммаризацию текста
Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
Проверить насколько хорошо она суммаризирует

2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import json
import random
import pandas as pd

from transformers import AutoTokenizer, EncoderDecoderModel

from datasets import load_metric

# Задание 1

Загрузка данных

In [ ]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.tx
     

In [ ]:
# функция чтения данных
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [ ]:
train_records = read_gazeta_records("/content/gazeta_train.txt")
val_records = read_gazeta_records("/content/gazeta_val.txt")
test_records = read_gazeta_records("/content/gazeta_test.tx")

In [ ]:
df = pd.DataFrame(test_records)
df.head(3)

,url,text,title,summary,date
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08


In [ ]:
df = df.drop(['url', 'date'], axis=1)

In [ ]:
df = df[:20]

Загрузка модели

In [ ]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.1 MB/s eta 0:00:00


In [ ]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [ ]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/3.47G [00:00<?, ?B/s]

Пример работы модели

In [ ]:
article_text = df['text'][1]

input_ids = tokenizer(
    [article_text],
    max_length=600,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)

print('Predicted summary: ', summary)
print('Real summary: ', df['summary'][1])

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted summary:  В Кремле прошла премьера новогоднего шоу «Танцуют все!» с участием более 300 артистов из одного региона. Зрителям рассказали, что в Бурятии сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов (семейских), эвенков.
Real summary:  25 и 26 февраля в Кремлевском дворце съездов праздновали Сагаалган — Восточный Новый год. Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.


Генерация краткого содержания для всего датасета

In [ ]:
data = df['text']

summaries = []

for row in data:
    input_ids = tokenizer(
        [row],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    summaries.append(summary)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
df['pred_summary'] = summaries
df.sample(5)

,text,title,summary,pred_summary
16,В 2019 году на Кубу прибыло рекордное число ту...,Дешевая экзотика: почему россияне уезжают на Кубу,Россияне стали чаще отдыхать на Кубе. За прошл...,В 2019 году на Кубу прибыло рекордное число ту...
10,Спрос на бензин в России в 2019 году упал на 1...,Дорогое удовольствие: почему снизился спрос на...,Рекордное снижение спроса на бензин зафиксиров...,Спрос на бензин в России в 2019 году упал на 1...
11,Американская юридическая компания FeganScott п...,Опасное излучение: на Apple и Samsung подали в...,Юридическая компания из США подала в суд на Ap...,Американская юридическая компания FeganScott п...
4,Депутат Верховной рады от партии «Слуга народа...,«Поддерживают Россию»: почему Киев не платит п...,Украина не должна выплачивать пенсии жителям Д...,Депутат Верховной рады от партии «Слуга народа...
7,Самые известные российские пранкеры Владимир «...,Именем Греты Тунберг: Вован и Лексус разыграли...,Пранкеры Вован и Лексус разыграли члена конгре...,Российские пранкеры Владимир «Вован» Кузнецов ...


In [ ]:
pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=c5809f2a53bc6bc93a29c7eefdec959c221bc3889ba413e165675c895a08efce
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score


In [ ]:
rouge_score = load_metric("rouge")

In [ ]:
predictions = df['pred_summary']
references = df['summary']

Rouge

In [ ]:
sum_scores = rouge_score.compute(
    predictions=[predictions], references=[references]
)
sum_scores

{'rouge1': AggregateScore(low=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), mid=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), high=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744)),
 'rouge2': AggregateScore(low=Score(precision=0.14634146341463414, recall=0.3157894736842105, fmeasure=0.19999999999999998), mid=Score(precision=0.14634146341463414, recall=0.3157894736842105, fmeasure=0.19999999999999998), high=Score(precision=0.14634146341463414, recall=0.3157894736842105, fmeasure=0.19999999999999998)),
 'rougeL': AggregateScore(low=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), mid=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), high=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744)),
 'rougeLsum': AggregateScore(low=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.419354838709677

# Задание 2

In [ ]:
model_name = "IlyaGusev/rubert_telegram_headlines"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, do_basic_tokenize=False, strip_accents=False)
model = EncoderDecoderModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/76.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Пример работы модели

In [ ]:
article_text = df['text'][1]

input_ids = tokenizer(
    [article_text],
    add_special_tokens=True,
    max_length=256,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=64,
    no_repeat_ngram_size=3,
    num_beams=10,
    top_p=0.95
)[0]

headline = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
print('Predicted headline: ', headline)
print('Real headline: ', df['title'][1])

Predicted headline:  В Кремле прошел самый лучший в России онлайн-концерт
Real headline:  «Люди в Бурятии очень талантливые»


Генерация заголовков для всего датасета

In [ ]:
data = df['text']

headlines = []

for row in data:
    input_ids = tokenizer(
        [row],
        add_special_tokens=True,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=64,
        no_repeat_ngram_size=3,
        num_beams=10,
        top_p=0.95
    )[0]
    
    headline = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    headlines.append(headline)

In [ ]:
df['pred_headline'] = headlines
df.sample(3)

,text,title,summary,pred_summary,pred_headline
15,Росстандарт готовит арсенал для проверок автоз...,Борьба с недоливом: заправки проверят скрытым ...,"Для борьбы с с недобросовестными АЗС, недолива...",Росстандарт готовит арсенал для проверок автоз...,"В Росстандарте рассказали, как проверить топли..."
12,На фоне эпидемии коронавируса китайские власти...,«Битва патогенов»: власти КНР выпустили игру п...,Правительство КНР совместно с государственной ...,На фоне эпидемии коронавируса китайские власти...,В Китае запустили мобильную игру о борьбе с ко...
1,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,В Кремле прошла премьера новогоднего шоу «Танц...,В Кремле прошел самый лучший в России онлайн-к...


In [ ]:
predictions = df['pred_headline']
references = df['title']

Rouge

In [ ]:
head_scores = rouge_score.compute(
    predictions=[predictions], references=[references]
)
head_scores

{'rouge1': AggregateScore(low=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), mid=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), high=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666)),
 'rouge2': AggregateScore(low=Score(precision=0.2222222222222222, recall=0.5, fmeasure=0.30769230769230765), mid=Score(precision=0.2222222222222222, recall=0.5, fmeasure=0.30769230769230765), high=Score(precision=0.2222222222222222, recall=0.5, fmeasure=0.30769230769230765)),
 'rougeL': AggregateScore(low=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), mid=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), high=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666)),
 'rougeLsum': AggregateScore(low=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), mid=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), high=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666))}